# Introduction
写在前面，在介绍CNN之前，首先记录本人自认为对CNN的一个重要理解：可以将CNN中的通道数**channel**理解为样本==特征数==。每个通道可以单独理解为样本的一个特征。<br>
经典CNN包括开山鼻祖LeNet、改进版更深层的AlexNet和VGG。<br>
- **LeNet** 是第一个CNN模型，包括卷积层、池化层、全连接层，卷积层后使用Sigmoid作为激活函数。<br>
- **AlexNet** 使用更大的卷积核、更多的通道数，使用LeRU作为激活函数，避免了梯度消失的问题。<br>
- **VGG** 提出通过重复使用简单的基础块构造深层模型，用连续多个小卷核的卷积层替代大卷积核的卷积层【保证感受野相似】。

# LeNet
